In [1]:
import os
from pandas import read_csv, read_excel, DataFrame
import sae
import gurobipy as gp
from gurobipy import GRB

import sys
sys.path.append('../')
import saedfsc

Here is a notional list of suppliers.

In [2]:
saedfsc.suppliers

,Name,Location,Rating
0,Supplier1,Location4,1.961931
1,Supplier2,Location1,4.843323
2,Supplier3,Location2,2.775760
3,Supplier4,Location3,2.757291
4,Supplier5,Location2,1.551448
5,Supplier6,Location3,1.926430
6,Supplier7,Location4,3.271909


In [4]:
partOptions = saedfsc.getPartOptionsWithSuppliers()

,Unnamed: 0_x,Material,Code,q,E,cost_per_kilogram,q (Ordering materials),Unnamed: 0_y,length,height,width,angle of attack,length*heigth*width (Ordering Wings),Supplier
909,4,Glass-filled nylon (35%),GFN,1600,14900000000,2.40,1600,45,0.30,0.200,0.05,0.157,0.003000,Supplier6
396,1,Nylon 6/6,N66,1150,2400000000,3.30,1150,180,0.55,0.200,2.54,0.785,0.279400,Supplier4
885,4,Glass-filled nylon (35%),GFN,1600,14900000000,2.40,1600,21,0.30,0.050,0.05,0.785,0.000750,Supplier5
1255,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,175,0.55,0.125,3.00,0.785,0.206250,Supplier1
1354,6,Glass-filled epoxy (35%),GFE,1900,25000000000,3.86,1900,58,0.55,0.200,0.05,0.785,0.005500,Supplier2
2591,11,"Cast Iron, grade 20",CI20,7150,100000000000,1.94,7150,215,0.55,0.550,3.00,0.785,0.907500,Supplier4
1642,7,Glass-filled polyester (35%),GFP,2000,15700000000,2.40,2000,130,0.05,0.550,2.54,0.628,0.069850,Supplier6
28,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,28,0.55,0.050,0.05,0.000,0.001375,Supplier1
2532,11,"Cast Iron, grade 20",CI20,7150,100000000000,1.94,7150,156,0.30,0.200,2.54,0.314,0.152400,Supplier1
1858,8,Aluminum 2045-T4,AT4,2700,73000000000,80.00,2700,130,0.05,0.550,2.54,0.628,0.069850,Supplier2


Here is a list of parts for the wings, along with the suppliers for each part.

In [5]:
partOptions['rearwing']

,Unnamed: 0_x,Material,Code,q,E,cost_per_kilogram,q (Ordering materials),Unnamed: 0_y,length,height,width,angle of attack,length*heigth*width (Ordering Wings),Supplier
909,4,Glass-filled nylon (35%),GFN,1600,14900000000,2.40,1600,45,0.30,0.200,0.05,0.157,0.003000,Supplier6
396,1,Nylon 6/6,N66,1150,2400000000,3.30,1150,180,0.55,0.200,2.54,0.785,0.279400,Supplier4
885,4,Glass-filled nylon (35%),GFN,1600,14900000000,2.40,1600,21,0.30,0.050,0.05,0.785,0.000750,Supplier5
1255,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,175,0.55,0.125,3.00,0.785,0.206250,Supplier1
1354,6,Glass-filled epoxy (35%),GFE,1900,25000000000,3.86,1900,58,0.55,0.200,0.05,0.785,0.005500,Supplier2
2591,11,"Cast Iron, grade 20",CI20,7150,100000000000,1.94,7150,215,0.55,0.550,3.00,0.785,0.907500,Supplier4
1642,7,Glass-filled polyester (35%),GFP,2000,15700000000,2.40,2000,130,0.05,0.550,2.54,0.628,0.069850,Supplier6
28,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,28,0.55,0.050,0.05,0.000,0.001375,Supplier1
2532,11,"Cast Iron, grade 20",CI20,7150,100000000000,1.94,7150,156,0.30,0.200,2.54,0.314,0.152400,Supplier1
1858,8,Aluminum 2045-T4,AT4,2700,73000000000,80.00,2700,130,0.05,0.550,2.54,0.628,0.069850,Supplier2


Here are the parts for the cabins.

In [7]:
partOptions['cabin']

,Unnamed: 0_x,Material,Code,q,E,cost_per_kilogram,q (Ordering materials),Unnamed: 0_y,length,height,width,thickness,length*heigth*width (Ordering Cabins),Supplier
995,5,S-glass epoxy (45%),SGE,1800,39500000000,3.76,1800,35,1.91,0.88,0.50,0.012,0.840400,Supplier1
464,2,Epoxy,EPY,1250,3350000000,2.76,1250,80,2.39,1.17,0.50,0.012,1.398150,Supplier7
107,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,107,1.91,1.17,0.79,0.012,1.765413,Supplier2
1677,8,Aluminum 2045-T4,AT4,2700,73000000000,80.00,2700,141,2.39,0.88,1.26,0.004,2.650032,Supplier4
1783,9,MgO,MGO,3600,205000000000,134.00,3600,55,1.52,0.88,0.79,0.008,1.056704,Supplier7
553,2,Epoxy,EPY,1250,3350000000,2.76,1250,169,1.52,1.17,2.00,0.008,3.556800,Supplier6
1326,6,Glass-filled epoxy (35%),GFE,1900,25000000000,3.86,1900,174,2.39,0.88,2.00,0.004,4.206400,Supplier5
177,0,Polyethylene (HDPE),HDPE,900,900000000,0.85,900,177,3.00,1.17,1.26,0.004,4.422600,Supplier4
1786,9,MgO,MGO,3600,205000000000,134.00,3600,58,1.91,1.17,0.50,0.008,1.117350,Supplier6
521,2,Epoxy,EPY,1250,3350000000,2.76,1250,137,3.00,0.66,1.26,0.012,2.494800,Supplier2


The partOptions dictionary can be used to query the parts lists (and suppliers) for any of the remaining subsystems.